In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot
from PIL import Image

In [3]:
! unzip "/content/gdrive/MyDrive/25K.zip" #

Streaming output truncated to the last 5000 lines.
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3940.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3949.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3954.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3957.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3964.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_397.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3970.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinoma_3975.png  
  inflating: NSCLC_Radiomics_50k_splitted2/validation/Large_cell_carcinoma/Large_cell_carcinom

In [4]:
path = "/content/NSCLC_Radiomics_50k_splitted2" #25K

In [5]:
train_ds = keras.utils.image_dataset_from_directory(
    f"{path}/train",
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(300, 300)

)

test_ds = keras.utils.image_dataset_from_directory(
    f"{path}/test",
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(300, 300)

)

val_ds = keras.utils.image_dataset_from_directory(
    f"{path}/validation",
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(300, 300)

)

Found 10000 files belonging to 5 classes.
Found 7500 files belonging to 5 classes.
Found 7500 files belonging to 5 classes.


In [6]:
# Define the new image size
new_image_size = (80, 80)

# Function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, new_image_size)
    return image, label

# Apply resizing to the datasets
train_ds = train_ds.map(resize_image)
test_ds = test_ds.map(resize_image)
val_ds = val_ds.map(resize_image)

In [7]:
# Normalize the pixel values
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)  # Convert to float32
    image = image / 255.0  # Normalize pixel values
    return image, label

# Apply preprocessing to the datasets
train_ds = train_ds.map(preprocess)
test_ds = test_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

# One-hot encode the labels
num_classes = 5  # Update with the actual number of classes
#train_ds = train_ds.map(lambda image, label: (image, tf.one_hot(label, num_classes)))
#test_ds = test_ds.map(lambda image, label: (image, tf.one_hot(label, num_classes)))
#val_ds = val_ds.map(lambda image, label: (image, tf.one_hot(label, num_classes)))

# Configure the datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [8]:
total_images = 51215
num_classes = 5
class_frequencies = {
    0: 6018,  # Adenocarcinoma
    1: 13655,  # Large_cell_carcinoma
    2: 5130,   # Normal
    3: 7643,   # Not_otherwise_specified
    4: 18769   # Squamous_cell_carcinoma
}

# Calculate class weights for imbalanced classes
class_weights = {}
total_samples = sum(class_frequencies.values())

for cls, num_samples in class_frequencies.items():
    weight = total_samples / (len(class_frequencies) * num_samples)
    class_weights[cls] = weight

print("Calculated Class Weights for Imbalanced Classes:", class_weights)

Calculated Class Weights for Imbalanced Classes: {0: 1.7020604852110335, 1: 0.7501281581838155, 2: 1.9966861598440546, 3: 1.3401805573727594, 4: 0.545740316479301}


In [9]:
# Define class names (modify this according to your dataset)
class_names = ["Adenocarcinoma", "Large_cell_carcinoma", "Normal", "Not_otherwise specified", "Squamous cell carcinoma"]
print(class_names)

['Adenocarcinoma', 'Large_cell_carcinoma', 'Normal', 'Not_otherwise specified', 'Squamous cell carcinoma']


In [10]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [11]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
from tensorflow.keras.preprocessing.image import img_to_array

# Load the model
@st.cache_data
def load_model():
    model = tf.keras.models.load_model('/content/gdrive/MyDrive/Webapp.hdf5')
    return model

def import_and_predict(image_data, model, class_names):
    # Preprocess the image
    img_array = img_to_array(image_data)
    img_array = tf.image.resize(img_array, [80, 80])  # Adjust the dimensions as needed
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

    # Make predictions
    predictions = model.predict(img_array)
    top_class = tf.argmax(predictions[0])
    confidence = tf.reduce_max(predictions[0])

    return top_class, confidence

# Title and description with lung icon
st.title("Lung 🫁 Cancer Classification Web App")

st.write(
    "This web app uses a Student model to classify lung cancer into categories. Upload an image and let's make a prediction!"
)

# Sidebar with information about the project
st.sidebar.title("About")
st.sidebar.info(
    "This is a simple web app for classifying lung cancer. It uses a Knowledge Distilled model trained on lung cancer data."
    "The model has been fine-tuned for high accuracy in detecting various types of lung cancer, including adenocarcinoma, large cell carcinoma, normal lung tissue, "
    "not otherwise specified, and squamous cell carcinoma."
    "To use the app, simply upload a PNG image of a lung, and the model will provide predictions along with confidence scores for different cancer types."
    "\n\n"
    "Please note that this web app is for educational and informational purposes only and should not be used for medical diagnosis. Consult with a healthcare professional for accurate medical assessments."
)

# File uploader and image display
file = st.file_uploader("Upload a lung image (PNG only)", type=["png"])

# Function to display prediction results with a progress bar
def display_prediction(top_class, confidence, class_names):
    st.subheader("Prediction Results:")
    st.write(f"Predicted Class: {class_names[top_class]}")
    st.write(f"Confidence: {confidence * 100:.2f}%")

if file is not None:
    image = Image.open(file)
    resized_image = image.resize((80, 80))  # Adjust the dimensions as needed
    st.sidebar.image(resized_image, caption="Uploaded Image", width=100)

    # Make predictions
    model = load_model()
    class_names = ["Adenocarcinoma", "Large_cell_carcinoma", "Normal", "Not_otherwise_specified", "Squamous_cell_carcinoma"]

    # Display class names
    st.subheader("Class Names:")
    for i, class_name in enumerate(class_names):
        st.write(f"{i + 1}. {class_name}")

    with st.spinner("Making prediction..."):
        top_class, confidence = import_and_predict(image, model, class_names)

    # Display top predictions
    display_prediction(top_class, confidence, class_names)
    #Done it


Writing app.py


In [12]:
!pip install pyngrok

In [13]:
!ngrok authtoken 2Z8NNyRD3BsuL1uMEvMKiFLoxFk_3cxYVo3gCpcnn3NR5JLxV #insert your autotoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [15]:
from pyngrok import ngrok
import subprocess

# Run the Streamlit app in the background
subprocess.Popen(["streamlit", "run", "app.py"])

# Wait for the Streamlit app to start (adjust the sleep time based on your app's startup time)
import time
time.sleep(10)

# Get the public URL using ngrok
ngrok_tunnel = ngrok.connect(8501)
public_url = ngrok_tunnel.public_url

# Print the public URL
print('Streamlit app is live at:', public_url)

Streamlit app is live at: https://fdb2-34-73-1-248.ngrok-free.app


In [16]:
!cat /content/nohup.out




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8502
  External URL: http://34.73.1.248:8502



In [17]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2023-12-06 12:40:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  26.8MB/s    in 0.5s    

2023-12-06 12:40:37 (26.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [18]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [19]:
get_ipython().system_raw('./ngrok http 8501 &')

In [20]:
!curl -s http://localhost:4040/api/tunnels | python3 -c 'import sys, json; print("Watch Streamlit live at: " + json.load(sys.stdin)["tunnels"][0]["public_url"])'
!streamlit run /content/app.py

Watch Streamlit live at: https://fdb2-34-73-1-248.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8503
  External URL: http://34.73.1.248:8503

  Stopping...
